<a href="https://colab.research.google.com/github/CS222-UIUC/course-project-movie-recommender/blob/hangook2/Hkang_Data_preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import sklearn as sk
import numpy as np
import pandas as pd
import tensorflow as ts
import matplotlib.pyplot as plt
import nltk
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [5]:
df = pd.read_csv("movies.csv")
df.head()

,id,title,genres,original_language,overview,popularity,production_companies,release_date,budget,revenue,runtime,status,tagline,vote_average,vote_count,credits,keywords,poster_path,backdrop_path,recommendations
0,631842,Knock at the Cabin,Horror-Mystery-Thriller,en,While vacationing at a remote cabin a young gi...,3422.537,Blinding Edge Pictures-Universal Pictures-Film...,2023-02-01,20000000.0,52000000.0,100.0,Released,Save your family or save humanity. Make the ch...,6.457,888.0,Dave Bautista-Jonathan Groff-Ben Aldridge-Kris...,based on novel or book-sacrifice-cabin-faith-e...,/dm06L9pxDOL9jNSK4Cb6y139rrG.jpg,/zWDMQX0sPaW2u0N2pJaYA8bVVaJ.jpg,1058949-646389-772515-505642-143970-667216-104...
1,646389,Plane,Action-Adventure-Thriller,en,After a heroic job of successfully landing his...,2618.646,MadRiver Pictures-Di Bonaventura Pictures-G-BA...,2023-01-12,25000000.0,51000000.0,107.0,Released,Survive together or die alone.,6.901,785.0,Gerard Butler-Mike Colter-Yoson An-Tony Goldwy...,pilot-airplane-philippines-held hostage-plane ...,/qi9r5xBgcc9KTxlOLjssEbDgO0J.jpg,/9Rq14Eyrf7Tu1xk0Pl7VcNbNh1n.jpg,505642-758769-864692-631842-1058949-925943-758...
2,315162,Puss in Boots: The Last Wish,Animation-Adventure-Comedy-Family,en,Puss in Boots discovers that his passion for a...,2589.341,DreamWorks Animation-Universal Pictures,2022-12-07,90000000.0,442000000.0,103.0,Released,Say hola to his little friends.,8.402,4357.0,Antonio Banderas-Salma Hayek-Harvey Guillén-Wa...,fairy tale-talking dog-spin off-aftercreditsst...,/kuf6dutpsT0vSVehic3EZIqkOBt.jpg,/b1Y8SUb12gPHCSSSNlbX4nB3IKy.jpg,536554-76600-816904-653851-899112-985335-83078...
3,505642,Black Panther: Wakanda Forever,Action-Adventure-Science Fiction,en,Queen Ramonda Shuri M’Baku Okoye and the Dora ...,2525.408,Marvel Studios,2022-11-09,250000000.0,858535561.0,162.0,Released,Forever.,7.338,3922.0,Letitia Wright-Lupita Nyong'o-Danai Gurira-Win...,loss of loved one-hero-sequel-superhero-based ...,/sv1xJUazXeYqALzczSZ3O6nkH75.jpg,/xDMIl84Qo5Tsu62c9DGWhmPI67A.jpg,436270-829280-76600-56969-312634-1037858-238-5...
4,956101,The Eighth Clause,Thriller,la,Kat and Borja appear to be a perfect couple bu...,2259.303,SDB Films-El Hombre Orquesta,2022-04-29,0.0,0.0,0.0,Released,NaN,4.600,10.0,Maite Perroni-Manuel Vega-Óscar Jaenada-Jessic...,NaN,/8tc8eMFAX2SDC1TRu987qFQy8Cl.jpg,/kLnqNE9Af5QHyvUxw8cDGhF1ilv.jpg,NaN


In [6]:
print(df.shape)

(8378, 20)


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8378 entries, 0 to 8377
Data columns (total 20 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   id                    8378 non-null   int64  
 1   title                 8378 non-null   object 
 2   genres                8277 non-null   object 
 3   original_language     8378 non-null   object 
 4   overview              8288 non-null   object 
 5   popularity            8377 non-null   float64
 6   production_companies  7735 non-null   object 
 7   release_date          8349 non-null   object 
 8   budget                8377 non-null   float64
 9   revenue               8377 non-null   float64
 10  runtime               8370 non-null   float64
 11  status                8377 non-null   object 
 12  tagline               5898 non-null   object 
 13  vote_average          8377 non-null   float64
 14  vote_count            8377 non-null   float64
 15  credits              

In [8]:
df.isnull().sum() # check null values

id                         0
title                      0
genres                   101
original_language          0
overview                  90
popularity                 1
production_companies     643
release_date              29
budget                     1
revenue                    1
runtime                    8
status                     1
tagline                 2480
vote_average               1
vote_count                 1
credits                  137
keywords                1195
poster_path               32
backdrop_path            359
recommendations         1347
dtype: int64

In [9]:
df.duplicated().sum()

0

In [10]:
vcounts = df[df['vote_count'].notnull()]['vote_count'].astype('int')
vaverages = df[df['vote_average'].notnull()]['vote_average'].astype('int')
avg = vaverages.mean()
avg

5.844932553420079

In [11]:
df = df.drop(["production_companies", "popularity", "budget", "revenue", "status", "recommendations", "runtime", "vote_average", "backdrop_path",'tagline'], axis = 1) #'poster_path'], axis=1)

In [12]:
df.drop_duplicates(inplace=True)

In [13]:
df.title.duplicated().sum()

254

In [14]:
df[["title", "release_date"]].duplicated().sum()# checking for release data with same movie title

15

In [15]:
df.drop_duplicates(subset=["title","release_date"], inplace=True)

In [16]:
df = df[df.vote_count >= 350].reset_index()

In [17]:
df.isnull().sum()

index                 0
id                    0
title                 0
genres                0
original_language     0
overview              0
release_date          0
vote_count            0
credits               1
keywords             75
poster_path           0
dtype: int64

In [18]:
df.fillna("", inplace = True) # replace the null values with ''
# We do have an option filling all the nulls with 0, but since they will have string values, I thought that na -> "" will be necessary

In [19]:
df

,index,id,title,genres,original_language,overview,release_date,vote_count,credits,keywords,poster_path
0,0,631842,Knock at the Cabin,Horror-Mystery-Thriller,en,While vacationing at a remote cabin a young gi...,2023-02-01,888.0,Dave Bautista-Jonathan Groff-Ben Aldridge-Kris...,based on novel or book-sacrifice-cabin-faith-e...,/dm06L9pxDOL9jNSK4Cb6y139rrG.jpg
1,1,646389,Plane,Action-Adventure-Thriller,en,After a heroic job of successfully landing his...,2023-01-12,785.0,Gerard Butler-Mike Colter-Yoson An-Tony Goldwy...,pilot-airplane-philippines-held hostage-plane ...,/qi9r5xBgcc9KTxlOLjssEbDgO0J.jpg
2,2,315162,Puss in Boots: The Last Wish,Animation-Adventure-Comedy-Family,en,Puss in Boots discovers that his passion for a...,2022-12-07,4357.0,Antonio Banderas-Salma Hayek-Harvey Guillén-Wa...,fairy tale-talking dog-spin off-aftercreditsst...,/kuf6dutpsT0vSVehic3EZIqkOBt.jpg
3,3,505642,Black Panther: Wakanda Forever,Action-Adventure-Science Fiction,en,Queen Ramonda Shuri M’Baku Okoye and the Dora ...,2022-11-09,3922.0,Letitia Wright-Lupita Nyong'o-Danai Gurira-Win...,loss of loved one-hero-sequel-superhero-based ...,/sv1xJUazXeYqALzczSZ3O6nkH75.jpg
4,7,937278,A Man Called Otto,Comedy-Drama,en,When a lively young family moves in next door ...,2022-12-28,540.0,Tom Hanks-Mariana Treviño-Rachel Keller-Manuel...,based on novel or book-suicide attempt-remake-...,/130H1gap9lFfiTF9iDrqNIkFvC9.jpg
...,...,...,...,...,...,...,...,...,...,...,...
4721,8366,901,City Lights,Comedy-Romance,en,In this sound-era silent film a tramp falls in...,1931-02-01,1818.0,Charlie Chaplin-Virginia Cherrill-Harry Myers-...,suicide attempt-operation-blindness and impair...,/bXNvzjULc9jrOVhGfjcc64uKZmZ.jpg
4722,8367,23082,The Invention of Lying,Comedy-Fantasy-Romance,en,Set in a world where the concept of lying does...,2009-09-13,1792.0,Ricky Gervais-Jennifer Garner-Jonah Hill-Louis...,lie-religion-lies,/clP8tDZeM9jgnqmu4VBClBDpLtS.jpg
4723,8369,277685,Unfriended,Horror-Thriller-Mystery,en,While video chatting one night six high school...,2014-10-22,3059.0,Shelley Hennig-Heather Sossaman-Renee Olstead-...,computer-bullying-revenge-internet-internet ch...,/8byPxREr3BqBNyyij5WHVSOVtYk.jpg
4724,8372,12516,Dreams,Fantasy-Drama,ja,A collection of magical tales based upon the a...,1990-05-11,361.0,Akira Terao-Mitsuko Baisho-Toshie Negishi-Miek...,life and death-artist-volcano-nuclear power pl...,/j0NQsYqnqvhTtBj7vhCa9iKT0NA.jpg


In [20]:
index = df[(df.genres == "") & (df.overview == "")].index
df.drop(index, inplace=True)

In [21]:
df.shape

(4726, 11)

In [22]:
df.head()

,index,id,title,genres,original_language,overview,release_date,vote_count,credits,keywords,poster_path
0,0,631842,Knock at the Cabin,Horror-Mystery-Thriller,en,While vacationing at a remote cabin a young gi...,2023-02-01,888.0,Dave Bautista-Jonathan Groff-Ben Aldridge-Kris...,based on novel or book-sacrifice-cabin-faith-e...,/dm06L9pxDOL9jNSK4Cb6y139rrG.jpg
1,1,646389,Plane,Action-Adventure-Thriller,en,After a heroic job of successfully landing his...,2023-01-12,785.0,Gerard Butler-Mike Colter-Yoson An-Tony Goldwy...,pilot-airplane-philippines-held hostage-plane ...,/qi9r5xBgcc9KTxlOLjssEbDgO0J.jpg
2,2,315162,Puss in Boots: The Last Wish,Animation-Adventure-Comedy-Family,en,Puss in Boots discovers that his passion for a...,2022-12-07,4357.0,Antonio Banderas-Salma Hayek-Harvey Guillén-Wa...,fairy tale-talking dog-spin off-aftercreditsst...,/kuf6dutpsT0vSVehic3EZIqkOBt.jpg
3,3,505642,Black Panther: Wakanda Forever,Action-Adventure-Science Fiction,en,Queen Ramonda Shuri M’Baku Okoye and the Dora ...,2022-11-09,3922.0,Letitia Wright-Lupita Nyong'o-Danai Gurira-Win...,loss of loved one-hero-sequel-superhero-based ...,/sv1xJUazXeYqALzczSZ3O6nkH75.jpg
4,7,937278,A Man Called Otto,Comedy-Drama,en,When a lively young family moves in next door ...,2022-12-28,540.0,Tom Hanks-Mariana Treviño-Rachel Keller-Manuel...,based on novel or book-suicide attempt-remake-...,/130H1gap9lFfiTF9iDrqNIkFvC9.jpg


In [23]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4726 entries, 0 to 4725
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   index              4726 non-null   int64  
 1   id                 4726 non-null   int64  
 2   title              4726 non-null   object 
 3   genres             4726 non-null   object 
 4   original_language  4726 non-null   object 
 5   overview           4726 non-null   object 
 6   release_date       4726 non-null   object 
 7   vote_count         4726 non-null   float64
 8   credits            4726 non-null   object 
 9   keywords           4726 non-null   object 
 10  poster_path        4726 non-null   object 
dtypes: float64(1), int64(2), object(8)
memory usage: 406.3+ KB


In [24]:
df.genres = df.genres.apply(lambda x: " ".join(x.split("-")))
df.keywords = df.keywords.apply(lambda x: " ".join(x.split("-")))
df.credits = df.credits.apply(lambda x: " ".join(x.replace(" ", "").split("-")[:5]))

In [25]:
df["tags"] =df.overview + " "+ df.genres + " "  +df.credits + " " +df.keywords + " " + df.original_language

In [26]:
df['tags']

0       While vacationing at a remote cabin a young gi...
1       After a heroic job of successfully landing his...
2       Puss in Boots discovers that his passion for a...
3       Queen Ramonda Shuri M’Baku Okoye and the Dora ...
4       When a lively young family moves in next door ...
                              ...                        
4721    In this sound-era silent film a tramp falls in...
4722    Set in a world where the concept of lying does...
4723    While video chatting one night six high school...
4724    A collection of magical tales based upon the a...
4725    Before being extradited to Africa to stand tri...
Name: tags, Length: 4726, dtype: object

In [27]:
df_new = df[["id", "title", "tags", 'poster_path']]

In [28]:
df_new

,id,title,tags,poster_path
0,631842,Knock at the Cabin,While vacationing at a remote cabin a young gi...,/dm06L9pxDOL9jNSK4Cb6y139rrG.jpg
1,646389,Plane,After a heroic job of successfully landing his...,/qi9r5xBgcc9KTxlOLjssEbDgO0J.jpg
2,315162,Puss in Boots: The Last Wish,Puss in Boots discovers that his passion for a...,/kuf6dutpsT0vSVehic3EZIqkOBt.jpg
3,505642,Black Panther: Wakanda Forever,Queen Ramonda Shuri M’Baku Okoye and the Dora ...,/sv1xJUazXeYqALzczSZ3O6nkH75.jpg
4,937278,A Man Called Otto,When a lively young family moves in next door ...,/130H1gap9lFfiTF9iDrqNIkFvC9.jpg
...,...,...,...,...
4721,901,City Lights,In this sound-era silent film a tramp falls in...,/bXNvzjULc9jrOVhGfjcc64uKZmZ.jpg
4722,23082,The Invention of Lying,Set in a world where the concept of lying does...,/clP8tDZeM9jgnqmu4VBClBDpLtS.jpg
4723,277685,Unfriended,While video chatting one night six high school...,/8byPxREr3BqBNyyij5WHVSOVtYk.jpg
4724,12516,Dreams,A collection of magical tales based upon the a...,/j0NQsYqnqvhTtBj7vhCa9iKT0NA.jpg


In [29]:
df_new.tags = df_new.tags.apply(lambda x:x.lower())

<ipython-input-29-99a8cb577326>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_new.tags = df_new.tags.apply(lambda x:x.lower())


In [30]:
df_new

,id,title,tags,poster_path
0,631842,Knock at the Cabin,while vacationing at a remote cabin a young gi...,/dm06L9pxDOL9jNSK4Cb6y139rrG.jpg
1,646389,Plane,after a heroic job of successfully landing his...,/qi9r5xBgcc9KTxlOLjssEbDgO0J.jpg
2,315162,Puss in Boots: The Last Wish,puss in boots discovers that his passion for a...,/kuf6dutpsT0vSVehic3EZIqkOBt.jpg
3,505642,Black Panther: Wakanda Forever,queen ramonda shuri m’baku okoye and the dora ...,/sv1xJUazXeYqALzczSZ3O6nkH75.jpg
4,937278,A Man Called Otto,when a lively young family moves in next door ...,/130H1gap9lFfiTF9iDrqNIkFvC9.jpg
...,...,...,...,...
4721,901,City Lights,in this sound-era silent film a tramp falls in...,/bXNvzjULc9jrOVhGfjcc64uKZmZ.jpg
4722,23082,The Invention of Lying,set in a world where the concept of lying does...,/clP8tDZeM9jgnqmu4VBClBDpLtS.jpg
4723,277685,Unfriended,while video chatting one night six high school...,/8byPxREr3BqBNyyij5WHVSOVtYk.jpg
4724,12516,Dreams,a collection of magical tales based upon the a...,/j0NQsYqnqvhTtBj7vhCa9iKT0NA.jpg


In [31]:
df_new.tags

0       while vacationing at a remote cabin a young gi...
1       after a heroic job of successfully landing his...
2       puss in boots discovers that his passion for a...
3       queen ramonda shuri m’baku okoye and the dora ...
4       when a lively young family moves in next door ...
                              ...                        
4721    in this sound-era silent film a tramp falls in...
4722    set in a world where the concept of lying does...
4723    while video chatting one night six high school...
4724    a collection of magical tales based upon the a...
4725    before being extradited to africa to stand tri...
Name: tags, Length: 4726, dtype: object

In [32]:
df_new.tags[0]

'while vacationing at a remote cabin a young girl and her two fathers are taken hostage by four armed strangers who demand that the family make an unthinkable choice to avert the apocalypse. with limited access to the outside world the family must decide what they believe before all is lost. horror mystery thriller davebautista jonathangroff benaldridge kristencui nikkiamuka based on novel or book sacrifice cabin faith end of the world apocalypse home invasion lgbt aftercreditsstinger adopted child adopted daughter shot on film gay same sex relationship religious symbolism en'

In [33]:
df_new.head()

,id,title,tags,poster_path
0,631842,Knock at the Cabin,while vacationing at a remote cabin a young gi...,/dm06L9pxDOL9jNSK4Cb6y139rrG.jpg
1,646389,Plane,after a heroic job of successfully landing his...,/qi9r5xBgcc9KTxlOLjssEbDgO0J.jpg
2,315162,Puss in Boots: The Last Wish,puss in boots discovers that his passion for a...,/kuf6dutpsT0vSVehic3EZIqkOBt.jpg
3,505642,Black Panther: Wakanda Forever,queen ramonda shuri m’baku okoye and the dora ...,/sv1xJUazXeYqALzczSZ3O6nkH75.jpg
4,937278,A Man Called Otto,when a lively young family moves in next door ...,/130H1gap9lFfiTF9iDrqNIkFvC9.jpg


In [34]:
prts = PorterStemmer()

In [35]:
def stem(text):
    y = []
    for i in text.split():
        y.append(prts.stem(i))
    
    return " ".join(y)    

In [36]:
df_new["tags"] = df_new["tags"].apply(stem)

<ipython-input-36-6e8e4450f3ce>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_new["tags"] = df_new["tags"].apply(stem)


In [37]:
countvec = CountVectorizer(stop_words="english",max_features=5000)

In [38]:
countvec

CountVectorizer(max_features=5000, stop_words='english')

In [39]:
vect = countvec.fit_transform(df_new["tags"]).toarray()

In [40]:
vect

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [41]:
countvec.get_feature_names_out()

array(['10', '1000', '11', ..., 'zoo', 'zooeydeschanel', 'zoëkravitz'],
      dtype=object)

In [42]:
sim = cosine_similarity(vect)

In [43]:
sim

array([[1.        , 0.06686508, 0.08711806, ..., 0.11291791, 0.09350492,
        0.02263394],
       [0.06686508, 1.        , 0.14213381, ..., 0.04386345, 0.        ,
        0.03077287],
       [0.08711806, 0.14213381, 1.        , ..., 0.03428963, 0.0745356 ,
        0.02405626],
       ...,
       [0.11291791, 0.04386345, 0.03428963, ..., 1.        , 0.04600437,
        0.        ],
       [0.09350492, 0.        , 0.0745356 , ..., 0.04600437, 1.        ,
        0.        ],
       [0.02263394, 0.03077287, 0.02405626, ..., 0.        , 0.        ,
        1.        ]])

In [44]:
sim.shape

(4726, 4726)

In [45]:
#now it is time for building recommendation system.

In [55]:
# by using how much the input is far from other data, we recommend the closest movie. -> similar movie

def recommend(movies):
    movie_index = df_new[df_new.title == movies].index[0]
    distances = sim[movie_index]
    movies_list = sorted(list(enumerate(distances)), reverse=True, key= lambda x:x[1])[1:6]
    
    for i in movies_list:
        print(df_new.iloc[i[0]].title)

In [56]:
#test 1

recommend("Batman")

Batman: Mask of the Phantasm
The Dark Knight
Batman & Robin
Batman Returns
Batman Begins


In [69]:
#test 2
while True:
  name = input("Plug in the movie that you want: ")
  if name.lower() == "done":
    break
  elif name in df_new["title"]:
    recommend(name)
  else:
    print("That movie is not in our program. Sorry, please choose a different movie")

Plug in the movie that you want: Batman


TypeError: ignored

In [70]:
print(df_new["title"])
print(type(df_new["title"].get(0)))
print(type(df_new["title"]))
print(df_new["title"].get(0) == "Knock at the Cabin")

0                   Knock at the Cabin
1                                Plane
2         Puss in Boots: The Last Wish
3       Black Panther: Wakanda Forever
4                    A Man Called Otto
                     ...              
4721                       City Lights
4722            The Invention of Lying
4723                        Unfriended
4724                            Dreams
4725                            Taxi 4
Name: title, Length: 4726, dtype: object
<class 'str'>
<class 'pandas.core.series.Series'>
True


In [74]:
print("Knock at the Cabin" in df_new["title"][0])

True


In [75]:
print(df_new["title"])

Knock at the Cabin
